<a href="https://colab.research.google.com/github/Danwhoosh/NovelOffice_Groq_ChatBot/blob/main/Novel_Office.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
chat_history = []

In [ ]:
!pip install -U langchain langchain-community

In [ ]:
!pip install gradio langchain langchain-community langchain-groq chromadb pdf2image pypdf sentence-transformers
!apt install poppler-utils -y


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
poppler-utils is already the newest version (22.02.0-2ubuntu0.8).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [ ]:
!apt install poppler-utils
!pip install pdf2image


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 35 not upgraded.
Need to get 186 kB of archives.
After this operation, 697 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.8 [186 kB]
Fetched 186 kB in 1s (356 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 126281 files and directories currently installed.)
Preparing to unpack .../poppler-utils_22.02.0-2ubuntu0.8_amd64.deb ...
Unpacking poppler-utils (22.02.0-2ubuntu0.8) ...
Setting up poppler-utils (22.02.0-2ubuntu0.8) ...
Processing triggers for man-db (2.10.2-1) ...


In [ ]:
!pip install langchain groq chromadb unstructured pypdf
!pip install -U langchain langchain-community groq chromadb unstructured pypdf

In [ ]:
from google.colab import files
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Upload your PDF
uploaded = files.upload()

# Get filename from uploaded file dictionary
pdf_file = list(uploaded.keys())[0]

# Load and chunk the document
loader = PyPDFLoader(pdf_file)
pages = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
docs = text_splitter.split_documents(pages)

print(f"✅ Loaded {len(docs)} chunks from {pdf_file}")


Saving FluidMechanics.pdf to FluidMechanics (1).pdf
✅ Loaded 2065 chunks from FluidMechanics (1).pdf


In [ ]:
!pip install sentence-transformers


In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma

# Free embedding model (no API key needed)
embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

# Create vector store from your document chunks
vectorstore = Chroma.from_documents(
    documents=docs,
    embedding=embedding_model,
    persist_directory="chroma_db"
)

vectorstore.persist()
print("✅ Vector store created and persisted.")



/tmp/ipython-input-4-4146879185.py:5: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  war

✅ Vector store created and persisted.


/tmp/ipython-input-4-4146879185.py:14: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectorstore.persist()


In [ ]:
# Extract title from first page or filename
first_page_text = pages[0].page_content.strip()
doc_title = pdf_file.replace(".pdf", "").replace("_", " ").title()

import re
title_match = re.search(r'(?i)(?:title[:\s]*)?(.{10,100})', first_page_text)
if title_match:
    doc_title = title_match.group(1).strip()

print(f"📘 Document Title: {doc_title}")


📘 Document Title: Genick Bar–Meir


In [ ]:
!pip install langchain-groq


In [ ]:
from langchain_groq import ChatGroq

In [ ]:
import os
from langchain_groq import ChatGroq
from langchain.chains import RetrievalQA
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings


os.environ["GROQ_API_KEY"] = "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"

# Reconnect vector store using Chroma and HuggingFace embeddings
embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
vectorstore = Chroma(
    persist_directory="chroma_db",
    embedding_function=embedding_model
)

# Loading Groq LLM (Mixtral)
llm = ChatGroq(
    model="llama3-70b-8192",
    temperature=0
)

# Build RetrievalQA Chain
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vectorstore.as_retriever(search_kwargs={"k": 3}),
    return_source_documents=True
)

print("✅ Groq LLM + Vector Store connected.")


✅ Groq LLM + Vector Store connected.


In [ ]:
from pdf2image import convert_from_path
from IPython.display import Image, display
import tempfile

def show_pdf_pages_as_images(pdf_path, page_numbers):
    # Convert pages to images
    with tempfile.TemporaryDirectory() as path:
        images = convert_from_path(pdf_path, dpi=150, first_page=min(page_numbers), last_page=max(page_numbers), output_folder=path)
        for i, img in enumerate(images):
            display(Markdown(f"### 🖼️ Page {page_numbers[i]}"))
            display(img)


In [ ]:
# Generate Summary of Document
from langchain.chains.question_answering import load_qa_chain

# summary chain
summary_chain = load_qa_chain(llm, chain_type="stuff")

# Using first 10 pages to summarize
summary_input_docs = pages[:10]
summary_prompt = f"Give a short summary of this document titled '{doc_title}' in 3-4 sentences."

summary_result = summary_chain.run(input_documents=summary_input_docs, question=summary_prompt)
print("📄 Summary Generated:")
from IPython.display import Markdown
display(Markdown(f"## 📘 Document Title: {doc_title}"))
display(Markdown(f"### 📝 Summary:\n{summary_result}"))



📄 Summary Generated:


## 📘 Document Title: Genick Bar–Meir

### 📝 Summary:
This document appears to be the front matter of a book titled "Basics of Fluid Mechanics" written by Genick Bar-Meir. The book is an open content resource that has been updated multiple times, and the author requests that readers update to the latest version. The book takes a practical approach to teaching fluid mechanics, covering topics such as material properties, integral analysis, and differential analysis. The author also mentions that the book has been plagiarized by others, including a professor at Northwestern University.

In [ ]:
from PyPDF2 import PdfReader


In [ ]:
!pip install PyPDF2


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 6.2 MB/s eta 0:00:00


In [ ]:
import gradio as gr
import os, tempfile
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_groq import ChatGroq
from langchain.chains import RetrievalQA
from pdf2image import convert_from_path
from PIL import Image

# 🔑 SET YOUR GROQ API KEY HERE
os.environ["GROQ_API_KEY"] = "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"

# 🔁 Global Variables
vectordb = None
qa_chain = None
doc_path = None
pages = []
chat_history = []

def extract_pdf_metadata(path):
    try:
        reader = PdfReader(path)
        info = reader.metadata
        title = info.title if info.title else "Unknown"
        author = info.author if info.author else "Unknown"
        return f"📘 **Title:** {title}\n👤 **Author:** {author}"
    except Exception as e:
        return f"⚠️ Could not extract metadata: {e}"

# 📄 Load PDF and embed
def load_pdf(uploaded_file, progress=gr.Progress()):
    global vectordb, qa_chain, doc_path, pages, chat_history
    chat_history.clear()

    progress(0, desc="⏳ Starting PDF processing...")

    # ✅ Use the uploaded file path
    doc_path = uploaded_file.name

    progress(0.1, desc="📄 Loading PDF...")
    loader = PyPDFLoader(doc_path)
    pages = loader.load()

    progress(0.3, desc="✂️ Splitting document...")
    splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    docs = splitter.split_documents(pages)

    progress(0.5, desc="🔗 Generating embeddings...")
    embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
    vectordb = Chroma.from_documents(docs, embeddings, persist_directory="chroma_store")

    progress(0.7, desc="💾 Saving vector store...")
    vectordb.persist()

    progress(0.8, desc="🤖 Loading Groq model...")
    llm = ChatGroq(model="llama3-70b-8192", temperature=0)

    progress(1.0, desc="✅ Finishing setup...")
    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        retriever=vectordb.as_retriever(search_kwargs={"k": 3}),
        return_source_documents=True
    )

    meta = extract_pdf_metadata(doc_path)
    return f"✅ PDF uploaded and processed.\n\n{meta}"




# 💬 Handle Query
def chat_with_pdf(query, eli5):
    if not qa_chain:
        return "⚠️ Please upload a PDF first.", [], []

    if eli5:
        query = f"Explain this like I'm 5: {query}"

    result = qa_chain(query)
    answer = result["result"]
    chat_history.append((query, answer))

    # 🧾 Build display history
    history_display = ""
    for i, (q, a) in enumerate(chat_history):
        history_display += f"**Q{i+1}:** {q}\n\n**A{i+1}:** {a}\n\n---\n"

    # 🖼️ Extract source pages
    images = []
    source_pages = set()
    for doc in result["source_documents"]:
        if hasattr(doc.metadata, 'get'):
            page_num = doc.metadata.get("page")
            if page_num is not None:
                source_pages.add(page_num + 1)

    if doc_path and source_pages:
        with tempfile.TemporaryDirectory() as path:
            for pg in sorted(source_pages):
                try:
                    imgs = convert_from_path(doc_path, dpi=150, first_page=pg, last_page=pg, output_folder=path)
                    if imgs:
                        images.append(imgs[0])
                except Exception as e:
                    print(f"Could not load page {pg}: {e}")

    return answer, history_display, images

# 🧹 Clear chat
def clear_chat():
    chat_history.clear()
    return "", "", []

# 🎨 Gradio Interface
with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown("## 📚 Groq-Powered Document Chatbot")
    with gr.Row():
        pdf_input = gr.File(label="📄 Upload PDF", file_types=[".pdf"])
        upload_btn = gr.Button("📤 Process PDF")

    eli5_toggle = gr.Checkbox(label="👶 Explain Like I'm 5")
    question_input = gr.Textbox(label="💬 Ask a question")
    ask_btn = gr.Button("Ask Groq")
    clear_btn = gr.Button("🧹 Clear Chat")

    answer_output = gr.Textbox(label="📋 Latest Answer", lines=4)
    chat_history_display = gr.Markdown()
    diagram_gallery = gr.Gallery(label="🖼️ Diagrams", columns=2, height="auto")

    upload_btn.click(load_pdf, inputs=[pdf_input], outputs=[answer_output])
    ask_btn.click(chat_with_pdf, inputs=[question_input, eli5_toggle],
                  outputs=[answer_output, chat_history_display, diagram_gallery])
    clear_btn.click(clear_chat, outputs=[answer_output, chat_history_display, diagram_gallery])

demo.launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://4ad01e2587f74b4ced.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
